In [2]:
import json

# Load the Yosys-generated JSON netlist
with open('synthesized_4x4.json', 'r') as f:
    netlist = json.load(f)

# Specify the top module manually (adjust the name as needed)
top_module_name = 'Multiplier_4x4'
if top_module_name not in netlist.get('modules', {}):
    raise KeyError(f"Module {top_module_name} not found in the netlist.")

module = netlist['modules'][top_module_name]

# Prepare a circuit dictionary with a set for nodes and a list for gates.
circuit = {"nodes": set(), "gates": []}

# Define a mapping from Yosys cell names to the gate types you want.
cell_mapping = {
    "$_AND_": "AND",
    "$_OR_": "OR",
    "$_NOT_": "NOT",
    "$_ANDNOT_": "ANDNOT",
    "$_ORNOT_": "ORNOT",
    "$_NAND_": "NAND",
    "$_NOR_": "NOR",
    "$_XOR_": "XOR",
    "$_XNOR_": "XNOR",
    # Add additional mappings if desired.
}

# Iterate over each cell in the top module.
for cell_name, cell in module.get("cells", {}).items():
    # Get the cell type and map it if necessary.
    yosys_type = cell["type"]
    gate_type = cell_mapping.get(yosys_type, yosys_type)

    # Build a list of nodes (nets) for this cell.
    gate_nodes = []
    for port, nets in cell.get("connections", {}).items():
        # In Yosys JSON, nets is usually a list (even for single connections).
        if isinstance(nets, list):
            # Assuming each port is connected to one net.
            net = nets[0]
        else:
            net = nets
        gate_nodes.append(net)
        # Add the net to our circuit's node set.
        circuit["nodes"].add(net)
    
    # Append this gate to the circuit list.
    circuit["gates"].append({"type": gate_type, "nodes": gate_nodes})

# Convert nodes from a set to a sorted list (for reproducibility)
circuit["nodes"] = sorted(list(circuit["nodes"]))

# Write the circuit dictionary to a JSON file
with open('circuit.json', 'w') as outfile:
    json.dump(circuit, outfile, indent=4)

print("Circuit has been written to circuit.json")

Circuit has been written to circuit.json


In [1]:
import Scripts.CustomScripts as CS
import Scripts.generate_weight_from_logic as GWL
import json

with open("circuit.json", 'r') as infile:
    circuit = json.load(infile)

# Preprocess the circuit to ensure that XOR and XNOR gates have the expected number of nodes.
for gate in circuit["gates"]:
    gate_type = gate["type"]
    # Set expected number of nodes based on gate type.
    if gate_type == "XOR":
        expected_nodes = 4  # 3 provided + 1 hidden
    elif gate_type == "XNOR":
        expected_nodes = 5  # 3 provided + 2 hidden
    else:
        continue  # Skip other gate types

    # While the gate doesn't have the expected number of nodes, add hidden nodes.
    while len(gate["nodes"]) < expected_nodes:
        # Create a unique hidden node name.
        hidden_node = f"hidden_{gate_type}_{circuit['gates'].index(gate)}_{len(gate['nodes'])}"
        gate["nodes"].append(hidden_node)
        # Also add the hidden node to the overall circuit's node list if it's not already there.
        if hidden_node not in circuit["nodes"]:
            circuit["nodes"].append(hidden_node)

h_total, J_total, nodes = GWL.generate_circuit_weights(circuit,SaveCircuit=False,CircuitName="Test")

{'4bFA': {'h': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'J': array([[ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  2., -1.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  2., -1.,  0.,  0.,
         0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  2., -1.,  0.,
         0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  2., -1.,
         0.,  0.,  0.,  1.],
       [-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2., -1.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2., -1.,  0.,  0.,
         0.,  1.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2., -1.,  0.,
         0.,  0.,  1.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2., -1.,
         0.,  0.,  0.,  1.],
       [ 2.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,
        -2.,  

ValueError: Gate type 'NOR' not defined in weight library.